## Holland profiles

In [1]:
# General
import glob
import os.path
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm

# Arrays & Displays
import xarray as xr
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
# from matplotlib.colors import Normalize
# from matplotlib.colors import ListedColormap
import matplotlib.cm as cm
import pandas as pd

# Data treatment
import dask as da
from dask.diagnostics import ProgressBar
import zarr
from scipy.interpolate import griddata
from scipy.signal import convolve2d
from scipy.signal import argrelextrema
from datetime import datetime

# Custom
# import functions as f

# Default parameters
mpl.rcParams.update({'font.size': 18})
mpl.rcParams['figure.figsize'] = (15, 10)
mpl.rcParams['axes.facecolor'] = 'white'
mpl.rcParams['figure.facecolor'] = 'white'

In [2]:
PRMS = {
    'save_figs':       True,
    'save_dir':        '/home/arthur/results/TCsLifeMonitFromObs/PRJCT_STEADY_STATE_CRIT/06_HOLLAND_PROFILES/',
    'ds_glob_path':    '/home/arthur/data/cyclobs/SAR_compilations_in_nc/SAR_centered_rotated_files_13052022_with_extra_params.nc',
}

### OPEN DATA
dsg = xr.open_dataset(PRMS['ds_glob_path'])

In [3]:
### FILTER
dsg = dsg.where(dsg.distance_to_coast > dsg.R17_1D * 1000, drop=True)
dsg = dsg.where(abs(dsg.lat_center) < 30, drop=True)
dsg = dsg.where(dsg.B_hol > 0.1, drop=True)
dsg = dsg.where(dsg.Vmax_1D > 20, drop=True)
print('Nb of storms: {}'.format(len(dsg.time)))

Nb of storms: 184


In [4]:
### FUNCTIONS
def coriolis(lat):
    '''Latitude must be in degrees.'''
    Omega = 7.2921e-5                            # Earth rotation vector
    fcor  = 2 * Omega * np.sin(lat * np.pi / 180) # Coriolis parameter at 20° latitude and assuming it's constant 
    return fcor

def holland_profile(r, lat, pn, pc, Vmin, Rmax, Vmax):
    '''We assume that rho is constant and equals 1.15 kg.m-3'''
    fcor  = abs(coriolis(lat))
    rho   = 1.15
    # B     = (Vmax ** 2) * rho * np.e / (pn - pc)
    B     = ((Vmax - Vmin) ** 2) * rho * np.e / (pn - pc)
    A     = Rmax ** B
    
    r     = r.astype(np.float) + 0.001 # To avoid dividing by zero
    V     = r * 0.
    V     = Vmin + np.sqrt(A * B * (pn - pc) * np.exp((-1) * A / (r ** B)) / (rho * r ** B) + (r ** 2 * fcor ** 2) / 4) - (r * fcor / 2)
    return V

In [13]:
rs = np.linspace(0, 500000, 50001)

for t in tqdm(range(len(dsg.time))):
    ds = dsg.isel(time=t)
    
    # Save
    if PRMS['save_figs']:
        plt.title('B = {:.1f}'.format(ds.B_hol), weight='bold')
        # SAR
        wsm = ds.wind_speed.mean(dim='theta')
        wss = ds.wind_speed.std(dim='theta')
        plt.plot(ds.rad, wsm, c='k', linewidth=2, label='SAR')
        plt.fill_between(ds.rad, wsm - wss, wsm + wss, color='gray', alpha=0.3)
        # Holland
        Vh = holland_profile(rs, float(abs(coriolis(ds.lat_center))), float(ds.pn_hol), float(ds.pc_hol), float(ds.Vmin_hol), float(ds.Rmax_hol), float(ds.Vmax_hol))
        plt.plot(rs, Vh, label='Holland')
        
        # Legend
        plt.xlabel('r (m)')
        plt.ylabel('v (m/s)')
        plt.legend();plt.grid()
        plt.grid(which='major', color='#DDDDDD', linewidth=0.8)
        plt.grid(which='minor', color='#EEEEEE', linestyle=':', linewidth=0.5);plt.minorticks_on()
        plt.ylim(5, 75);plt.xlim(0, 500000)
        
        # Save
        plt.savefig(PRMS['save_dir'] + os.path.splitext(ds.sat_file.item().item())[0]);plt.clf()

100%|█████████████████████████████████████████| 184/184 [00:39<00:00,  4.70it/s]


<Figure size 1500x1000 with 0 Axes>